In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('EURUSD_data')

# Select features and target
features = data[['Open', 'High', 'Low', 'Close', 'Adj Close']]
target = data['Close']

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(features)

# Create sequences of data for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 3])  # Use the 'Close' price as target
    return np.array(X), np.array(y)

seq_length = 60  # Using 60 days of data to predict the next day's price
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

### Step 1: Define the Evaluation Function

In [11]:
import numpy as np
from geneticalgorithm import geneticalgorithm as ga
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# Define the evaluation function
# Define the evaluation function
def evaluate_lstm(params):
    # Decode the parameters
    units_layer_1 = units_layer_1_values[int(params[0])]
    units_layer_2 = units_layer_2_values[int(params[1])]
    dropout_rate = dropout_rate_values[int(params[2])]
    learning_rate = learning_rate_values[int(params[3])]

    # Build the GRU model
    model = Sequential()
    model.add(GRU(units=units_layer_1, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(GRU(units=units_layer_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)  # Using fewer epochs for quicker evaluation

    # Predict on test data
    predicted_prices = model.predict(X_test)
    
    # Evaluate the model's performance
    mse = mean_squared_error(y_test, predicted_prices)
    
    return mse
  # The GA will minimize this value

### Step 2: Define the Genetic Algorithm Configuration

In [13]:
import numpy as np
from geneticalgorithm import geneticalgorithm as ga
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# Specific values for the parameters
units_layer_1_values = [50, 100, 150, 200]  # Possible units for the first GRU layer
units_layer_2_values = [20, 60, 100]  # Possible units for the second GRU layer
dropout_rate_values = [0.1,0.3,0.5]  # Possible dropout rates (in range [0, 1))
learning_rate_values = [0.0001, 0.005, 0.01]  # Possible learning rates

# Convert these specific values into indices for the GA
varbound = np.array([
    [0, len(units_layer_1_values) - 1],
    [0, len(units_layer_2_values) - 1],
    [0, len(dropout_rate_values) - 1],
    [0, len(learning_rate_values) - 1]
])

# Initialize the Genetic Algorithm
modelGA = ga(function=evaluate_lstm, dimension=4, variable_type='int', variable_boundaries=varbound, function_timeout=120)

# Run the Genetic Algorithm
modelGA.run()

# Best parameters found by GA
best_params_indices = modelGA.output_dict['variable']
best_mse = modelGA.output_dict['function']

# Decode the best parameters
best_params = [
    units_layer_1_values[int(best_params_indices[0])],
    units_layer_2_values[int(best_params_indices[1])],
    dropout_rate_values[int(best_params_indices[2])],
    learning_rate_values[int(best_params_indices[3])]
]

print('Best parameters found by genetic algorithm:', best_params)
print("Best Mean Squared Error:", best_mse)

16/16 [==============================] - 1s 9ms/step


16/16 [==============================] - 0s 9ms/step


16/16 [==============================] - 1s 17ms/step


16/16 [==============================] - 1s 14ms/step


16/16 [==============================] - 1s 13ms/step


...

16/16 [==============================] - 1s 4ms/step


16/16 [==============================] - 1s 11ms/step


16/16 [==============================] - 1s 4ms/step


16/16 [==============================] - 1s 14ms/step


16/16 [==============================] - 1s 4ms/step
